# qip+GI-ICA-k4+SINR

In [536]:
library(numDeriv)
library(pracma)
library(matrixcalc)
library(MASS)

In [554]:
fastICA_personalImplementation <- function(X, nonLinearityFunc= function(x) x^4, numComps=1){
    
    nonLinearityFuncDeriv <- function(x){
        
        return(grad(nonLinearityFunc, x))
        
    }
    
    wUpdate <- function(w_kMinusOne){
        
        wkMinusOneTXs <- XCentered %*% w_kMinusOne
                          
        expectationTerm_g <- (t(XCentered) %*% nonLinearityFunc(wkMinusOneTXs))/dim(X)[1]      
        
        expectationTerm_gPrime <- mean(nonLinearityFuncDeriv(wkMinusOneTXs))
    
        w_k <- C_pinv %*% expectationTerm_g-expectationTerm_gPrime*w_kMinusOne

        w_k <-  w_k/sqrt(as.numeric(t(w_k) %*% C %*% w_k))
        
        return(w_k)
        
    }
    
    XMean <- sapply(colMeans(X), FUN=function(x) rep(x, dim(X)[1]))
    
    XCentered <- X-XMean
    
    C <- cov(XCentered)
                          
    C_pinv <- pinv(C)
    
    varsForWInit <- array(rnorm(dim(X)[2]*numComps), dim=c(numComps, dim(X)[2]))
    
    lambdaSquared <- t(sapply(rowSums(varsForWInit^2),FUN = function(x) rep(x, dim(X)[2])))
    
    w_k <- varsForWInit/sqrt(lambdaSquared)
                          
    for (i in 1:100){
        
       for (coord in 1:dim(w_k)[1]){

           w_k[coord, ] <-  wUpdate(w_kMinusOne=w_k[coord, ])
           
           if (coord==1){
               
               next
               
           }else{
               
            previousConsts <- array(apply(w_k[1:(coord-1), , drop=FALSE], MARGIN=1, 
                                    FUN= function(x) w_k[coord, , drop=FALSE] %*% C %*% x ), dim=c(coord-1, 1))

            toBeSubtracted <- t(w_k[1:(coord-1), , drop=FALSE]) %*% previousConsts 
                                          
            w_k[coord, ] <- w_k[coord, ]- toBeSubtracted
                                          
            w_k[coord, ] <- w_k[coord, ]/sqrt(as.numeric(t(w_k[coord,]) %*% C %*% w_k[coord, ]))
            
               
           }
           
           
           }
        
    }                   
    
                                          
    signalEstimates <- XCentered %*% t(w_k)                          
                                          
    return(list(U=w_k, S=signalEstimates+ XMean %*% t(w_k)))
                                                 
}

In [555]:
indepReals <- array(rgamma(4*10000, shape=1, scale=1), dim=c(4, 10000))


UTrue <- gramSchmidt(array(runif(dim(indepReals)[1]^2, min=-1, max=1), 
                               dim=rep(dim(indepReals)[1], 2)))$Q

In [556]:
observedData <- t(t(UTrue) %*% indepReals)

In [557]:
icaData <- fastICA_personalImplementation(X=observedData, nonLinearityFunc=function(x) tanh(x), numComps=4)

# FastICA for Noisy Data

In [ ]:
noisyFastICA <- function(X, Sigma, nonLinearityFunc= function(x) x^4, numComps=1){
    
    nonLinearityFuncDeriv <- function(x){
        
        return(grad(nonLinearityFunc, x))
        
    }
    
    wUpdate <- function(w_kMinusOne){
        
        wkMinusOneTXs <- XTilde %*% w_kMinusOne
                          
        expectationTerm_g <- (t(XTilde) %*% nonLinearityFunc(wkMinusOneTXs))/dim(X)[1]      
        
        expectationTerm_gPrime <- mean(nonLinearityFuncDeriv(wkMinusOneTXs))
    
        w_k <- expectationTerm_g-expectationTerm_gPrime*(diag(rep(1, dim(SigmaTilde)[1]))+ SigmaTilde) %*% w_kMinusOne

        w_k <-  w_k/sqrt(as.numeric(t(w_k) %*% C %*% w_k))
        
        return(w_k)
        
    }
    
    XCentered <- X-sapply(colMeans(X), FUN=function(x) rep(x, dim(X)[1]))
    
    C <- cov(XCentered)
                          
    CMinusSigma <- C-Sigma ## Must be Invertible!! 
                          
    CMinusSigmaMinusHalf <- matrix.power(CMinusSigma, -.5)
                          
    SigmaTilde <- CMinusSigmaMinusHalf %*% Sigma %*% CMinusSigmaMinusHalf
                          
    XTilde <- CMinusSigmaMinusHalf %*% XCentered
    
    varsForWInit <- array(rnorm(dim(X)[2]*numComps), dim=c(numComps, dim(X)[2]))
    
    lambdaSquared <- t(sapply(rowSums(varsForWInit^2),FUN = function(x) rep(x, dim(X)[2])))
    
    w_k <- varsForWInit/sqrt(lambdaSquared)
                          
    for (i in 1:100){
        
       for (coord in 1:dim(w_k)[1]){

           w_k[coord, ] <-  wUpdate(w_kMinusOne=w_k[coord, ])
           
           if (coord==1){
               
               next
               
           }else{
               
            previousConsts <- array(apply(w_k[1:(coord-1), , drop=FALSE], MARGIN=1, 
                                    FUN= function(x) w_k[coord, , drop=FALSE] %*% C %*% x ), dim=c(coord-1, 1))

            toBeSubtracted <- t(w_k[1:(coord-1), , drop=FALSE]) %*% previousConsts 
                                          
            w_k[coord, ] <- w_k[coord, ]- toBeSubtracted
                                          
            w_k[coord, ] <- w_k[coord, ]/sqrt(as.numeric(t(w_k[coord,]) %*% C %*% w_k[coord, ]))
            
               
           }
           
           
           }
        
    }                   
    
                                          
    signalEstimates <- XTilde %*% t(w_k)    ### Signals must be estimated from XTilde!                       
                                          
    return(list(U=w_k, S=signalEstimates)) 
                                                 
}